In [1]:
import pandas as pd
import statistics as st
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from sklearn import preprocessing
torch.manual_seed(42)

# Data Preprocessing

In [2]:
le=preprocessing.LabelEncoder()
df = pd.read_csv('train.csv')
df=df.dropna()
# df['birth_year']=df['birth_year'].apply(lambda x:year(x))
df['join_year']=pd.DatetimeIndex(df['join_date']).year.values
df['join_month']=pd.DatetimeIndex(df['join_date'],dayfirst=True).month.values
df['occupation_code']=le.fit_transform(df['occupation_code'])
df['age']=df['join_year']-df['birth_year']
del(df['join_date'])
# del(df['join_year'])
# del(df['birth_year'])
df=df[['ID']+list(df.columns.values[-3:])+list(df.columns.values[1:-3])]


In [3]:
df

,ID,join_year,join_month,age,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,...,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,4WKQSBB,2019,2,32,F,M,1987,1X1H,17,T4MS,...,0,0,0,1,0,0,0,0,0,0
1,CP5S02H,2019,6,38,F,M,1981,UAOD,17,T4MS,...,0,0,0,1,0,0,0,0,0,0
2,2YKDILJ,2013,6,22,M,U,1991,748L,176,90QI,...,0,0,0,0,0,0,0,0,0,1
3,2S9E81J,2019,8,29,M,M,1990,1X1H,76,56SI,...,0,0,0,1,0,0,0,0,0,0
4,BHDYVFT,2019,8,29,M,M,1990,748L,148,T4MS,...,0,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29127,8SBA93U,2019,9,45,F,M,1974,748L,17,T4MS,...,0,0,0,1,0,0,0,0,0,0
29128,Y73ETTO,2018,11,31,M,M,1987,XX25,170,90QI,...,0,0,0,1,0,0,0,0,0,0
29129,MVXG2LC,2019,2,34,M,M,1985,BOAS,44,T4MS,...,0,0,0,1,0,0,0,0,0,0
29130,BUQ8DQN,2019,7,30,M,M,1989,UAOD,17,T4MS,...,0,0,0,1,0,0,0,0,0,0


In [4]:
# join_year = list(df['join_year'].unique())
# join_month = list(df['join_month'].unique())
age = list(df['age'].unique())
sex=list(df['sex'].unique())
marital_status=list(df['marital_status'].unique())
branch_code=list(df['branch_code'].unique())
occupation_code=list(df['occupation_code'].unique())
occupation_category_code=list(df['occupation_category_code'].unique())
# birth_year=list(df['birth_year'].unique())


In [5]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

In [6]:
#train_x=torch.tensor([list(df['ID'].unique()).index(item) for item in list(df['ID'])]).to(device)
df

,ID,join_year,join_month,age,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,...,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,4WKQSBB,2019,2,32,F,M,1987,1X1H,17,T4MS,...,0,0,0,1,0,0,0,0,0,0
1,CP5S02H,2019,6,38,F,M,1981,UAOD,17,T4MS,...,0,0,0,1,0,0,0,0,0,0
2,2YKDILJ,2013,6,22,M,U,1991,748L,176,90QI,...,0,0,0,0,0,0,0,0,0,1
3,2S9E81J,2019,8,29,M,M,1990,1X1H,76,56SI,...,0,0,0,1,0,0,0,0,0,0
4,BHDYVFT,2019,8,29,M,M,1990,748L,148,T4MS,...,0,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29127,8SBA93U,2019,9,45,F,M,1974,748L,17,T4MS,...,0,0,0,1,0,0,0,0,0,0
29128,Y73ETTO,2018,11,31,M,M,1987,XX25,170,90QI,...,0,0,0,1,0,0,0,0,0,0
29129,MVXG2LC,2019,2,34,M,M,1985,BOAS,44,T4MS,...,0,0,0,1,0,0,0,0,0,0
29130,BUQ8DQN,2019,7,30,M,M,1989,UAOD,17,T4MS,...,0,0,0,1,0,0,0,0,0,0


# Model Building

In [7]:
#0.09 model

class MatrixFactorization(torch.nn.Module):
    
    def __init__(self, n_items, n_factors=20):
        super().__init__()
        self.item_factors = torch.nn.Embedding(n_items, n_factors)
        self.item_biases = torch.nn.Embedding(n_items,n_factors)
        torch.nn.init.xavier_uniform_(self.item_factors.weight)
        self.item_biases.weight.data.fill_(1.)
        
        self.sex_factors = torch.nn.Embedding(len(sex), n_factors)
        self.sex_biases = torch.nn.Embedding(len(sex), n_factors)
        torch.nn.init.xavier_uniform_(self.sex_factors.weight)
        self.sex_biases.weight.data.fill_(1.)
        
        self.age_factors = torch.nn.Embedding(len(age), n_factors)
        self.age_biases = torch.nn.Embedding(len(age), n_factors)
        torch.nn.init.xavier_uniform_(self.age_factors.weight)
        self.age_biases.weight.data.fill_(1.)
        
#         self.join_year_factors = torch.nn.Embedding(len(join_year), n_factors)
#         self.join_year_biases = torch.nn.Embedding(len(join_year), n_factors)
#         torch.nn.init.xavier_uniform_(self.join_year_factors.weight)
#         self.join_year_biases.weight.data.fill_(1.)
        
#         self.join_month_factors = torch.nn.Embedding(len(join_month), n_factors)
#         self.join_month_biases = torch.nn.Embedding(len(join_month), n_factors)
#         torch.nn.init.xavier_uniform_(self.join_month_factors.weight)
#         self.join_month_biases.weight.data.fill_(1.)
        
        self.marital_status_factors = torch.nn.Embedding(len(marital_status), n_factors)
        self.marital_status_biases = torch.nn.Embedding(len(marital_status), n_factors)
        torch.nn.init.xavier_uniform_(self.marital_status_factors.weight)
        self.marital_status_biases.weight.data.fill_(1.)
        
        self.branch_code_factors = torch.nn.Embedding(len(branch_code), n_factors)
        self.branch_code_biases = torch.nn.Embedding(len(branch_code), n_factors)
        torch.nn.init.xavier_uniform_(self.branch_code_factors.weight)
        self.branch_code_biases.weight.data.fill_(1.)
        
#         self.occupation_code_factors = torch.nn.Embedding(len(occupation_code), n_factors)
#         self.occupation_code_biases = torch.nn.Embedding(len(occupation_code), n_factors)
#         torch.nn.init.xavier_uniform_(self.occupation_code_factors.weight)
#         self.occupation_code_biases.weight.data.fill_(1.)
        
        self.occupation_category_code_factors = torch.nn.Embedding(len(occupation_category_code), n_factors)
        self.occupation_category_code_biases = torch.nn.Embedding(len(occupation_category_code), n_factors)
        torch.nn.init.xavier_uniform_(self.occupation_category_code_factors.weight)
        self.occupation_category_code_biases.weight.data.fill_(1.)
        
#         self.birth_year_factors = torch.nn.Embedding(len(birth_year), n_factors)
#         self.birth_year_biases = torch.nn.Embedding(len(birth_year), n_factors)
#         torch.nn.init.xavier_uniform_(self.birth_year_factors.weight)
#         self.birth_year_biases.weight.data.fill_(1.)
        

#         self.fc1 = nn.Linear(8192,2048)
#         self.fc2 = nn.Linear(4096,2048)
#         self.fc3 = nn.Linear(2048,1024)
#         self.fc4 = nn.Linear(1024,512)
#         self.fc5 = nn.Linear(512,128)
#         self.fc6 = nn.Linear(128,64)
#         self.fc7 = nn.Linear(64,32)
#         self.fc8 = nn.Linear(32,21)
        
    def forward(self, inpt):
        
        array=inpt[0].T
        sex_i = torch.tensor([sex.index(item) for item in array[4]],dtype=torch.long).to(device)
        age_i = torch.tensor([age.index(item) for item in array[3]],dtype=torch.long).to(device)
        marital_status_i = torch.tensor([marital_status.index(item) for item in array[5]],dtype=torch.long).to(device)
        branch_code_i = torch.tensor([branch_code.index(item) for item in array[7]],dtype=torch.long).to(device)
        occupation_code_i = torch.tensor([occupation_code.index(item) for item in array[8]],dtype=torch.long).to(device)
        occupation_category_code_i = torch.tensor([occupation_category_code.index(item) for item in array[9]],dtype=torch.long).to(device)
#         birth_year_i = torch.tensor([birth_year.index(item) for item in array[5]],dtype=torch.long).to(device)
#         join_month_i = torch.tensor([join_month.index(item) for item in array[2]],dtype=torch.long).to(device)
#         join_year_i = torch.tensor([join_year.index(item) for item in array[1]],dtype=torch.long).to(device)
        

        
#         pred += torch.mm(self.sex_factors(sex_i), model.item_factors(torch.tensor(list(np.arange(21)),dtype=torch.long).to(device)).T)
#         pred += torch.mm(self.marital_status_factors(marital_status_i), model.item_factors(torch.tensor(list(np.arange(21)),dtype=torch.long).to(device)).T)
#         pred += torch.mm(self.branch_code_factors(branch_code_i), model.item_factors(torch.tensor(list(np.arange(21)),dtype=torch.long).to(device)).T)
#         pred += torch.mm(self.occupation_category_code_factors(occupation_category_code_i), model.item_factors(torch.tensor(list(np.arange(21)),dtype=torch.long).to(device)).T)
#         pred += torch.mm(self.birth_year_factors(birth_year_i), model.item_factors(torch.tensor(list(np.arange(21)),dtype=torch.long).to(device)).T)
        user_biases = self.age_biases(age_i) + self.sex_biases(sex_i) + self.marital_status_biases(marital_status_i) + self.branch_code_biases(branch_code_i) + self.occupation_category_code_biases(occupation_category_code_i)
        item_biases = self.item_biases(torch.tensor(list(np.arange(21)),dtype=torch.long).to(device)).T
        
        user_factors = self.age_factors(age_i)+ self.sex_factors(sex_i)+self.marital_status_factors(marital_status_i)+self.branch_code_factors(branch_code_i)+self.occupation_category_code_factors(occupation_category_code_i)
        item_factors = self.item_factors(torch.tensor(list(np.arange(21)),dtype=torch.long).to(device)).T
        
        user = user_factors 
        item = item_factors 
        pred = torch.mm(user_biases,item_biases)
        pred += torch.mm(user,item )
        
#         pred = abs(F.normalize(self.fc1(pred)))
# #         pred = F.normalize(self.fc2(pred))
#         pred = abs(F.normalize(self.fc3(pred)))
#         pred = F.normalize(self.fc4(pred))
#         pred = abs(F.normalize(self.fc5(pred)))
#         pred = self.fc6(pred)
#         pred = self.fc7(pred)
#         pred = self.fc8(pred)
        
        pred = abs(F.normalize(pred))

        
        return pred
    
nfactor = 750
model = MatrixFactorization(21, n_factors=nfactor).to(device)

In [8]:
df.head()

,ID,join_year,join_month,age,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,...,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,4WKQSBB,2019,2,32,F,M,1987,1X1H,17,T4MS,...,0,0,0,1,0,0,0,0,0,0
1,CP5S02H,2019,6,38,F,M,1981,UAOD,17,T4MS,...,0,0,0,1,0,0,0,0,0,0
2,2YKDILJ,2013,6,22,M,U,1991,748L,176,90QI,...,0,0,0,0,0,0,0,0,0,1
3,2S9E81J,2019,8,29,M,M,1990,1X1H,76,56SI,...,0,0,0,1,0,0,0,0,0,0
4,BHDYVFT,2019,8,29,M,M,1990,748L,148,T4MS,...,0,0,0,0,0,0,1,1,0,0


# Trainig the Model

In [9]:
batch_size = 12000
epochs=120

criterion = nn.BCELoss()
for i in tqdm(range(epochs)):
    seq=torch.randperm(29130)
    train_x = df.iloc[:,:10].values
    train_y=df.iloc[:,10:].values
    cum_loss=0
    count=0
    
    for i in range(0,len(train_x),batch_size):
        reg=torch.tensor(0.).to(device)
        for param in model.parameters():
            reg+=(param**2).sum()
        optimizer = torch.optim.Adam(model.parameters(),lr=0.005) 
        train_batch_y = torch.tensor(train_y[i:i+batch_size],dtype=torch.float32).to(device)
        output=model((train_x[i:i+batch_size],train_batch_y))
        
        loss = criterion(output, train_batch_y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        cum_loss+=loss
        count+=1
        
    print(cum_loss/count)

  1%|          | 1/120 [00:00<01:26,  1.38it/s]

tensor(0.3839, device='cuda:0', grad_fn=<DivBackward0>)


  2%|▏         | 2/120 [00:01<01:15,  1.57it/s]

tensor(0.3810, device='cuda:0', grad_fn=<DivBackward0>)


  2%|▎         | 3/120 [00:01<01:08,  1.72it/s]

...

tensor(0.1555, device='cuda:0', grad_fn=<DivBackward0>)


 99%|█████████▉| 119/120 [00:51<00:00,  2.23it/s]

tensor(0.1557, device='cuda:0', grad_fn=<DivBackward0>)


100%|██████████| 120/120 [00:51<00:00,  2.31it/s]

tensor(0.1559, device='cuda:0', grad_fn=<DivBackward0>)


In [10]:
# torch.save({
#             'model_state_dict': model.state_dict(),
#             'optimizer_state_dict': optimizer.state_dict(), 
#             'loss': loss
#             }, 'model_v1.pth.tar')

In [11]:
# optimizer = torch.optim.AdamW(model.parameters(), lr=0.001) 
# cp = torch.load('model_v1.pth.tar')

In [12]:
# model.load_state_dict(cp['model_state_dict'])
# optimizer.load_state_dict(cp['optimizer_state_dict'])

In [13]:
dt = pd.read_csv('test.csv')
# df['birth_year']=df['birth_year'].apply(lambda x:year(x))
dt['join_year']=pd.DatetimeIndex(dt['join_date']).year.values.astype(int)
dt['join_month']=pd.DatetimeIndex(dt['join_date'],dayfirst=True).month.values.astype(int)
dt['occupation_code']=le.fit_transform(dt['occupation_code'])
del(dt['join_date'])

dt.loc[~dt['join_year'].isin(df['join_year']),'join_year']=int(st.mean(dt[dt['join_year'].isin(df['join_year'])]['join_year']))
dt.loc[~dt['birth_year'].isin(df['birth_year']),'birth_year']=int(st.mean(dt[dt['birth_year'].isin(df['birth_year'])]['birth_year']))
dt.loc[~dt['join_month'].isin(df['join_month']),'join_month']=np.nan
dt=dt.fillna(dt.groupby('join_year').transform('mean').astype(int))
dt['join_month']=dt['join_month'].astype(int)
dt['age']=dt['join_year']-dt['birth_year']
dt=dt[['ID']+list(dt.columns.values[-3:])+list(dt.columns.values[1:-3])]

In [14]:
dt

,ID,join_year,join_month,age,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,...,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,F86J5PC,2018,12,34,M,M,1984,94KC,79,90QI,...,0,0,0,0,0,0,0,0,0,0
1,H6141K3,2019,10,23,M,M,1996,1X1H,103,90QI,...,0,0,0,1,0,0,0,0,0,0
2,RBAYUXZ,2020,1,52,F,W,1968,UAOD,18,T4MS,...,0,0,0,1,0,0,0,0,0,0
3,KCBILBQ,2019,2,30,M,M,1989,94KC,18,T4MS,...,0,0,0,0,0,0,0,0,0,0
4,LSEC1ZJ,2020,2,38,F,M,1982,UAOD,3,T4MS,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0GMU5UH,2019,4,33,F,M,1986,748L,18,T4MS,...,0,0,0,0,1,0,1,0,0,0
9996,I9W11CD,2019,8,39,M,M,1980,UAOD,3,T4MS,...,0,0,0,0,0,0,0,0,0,0
9997,42WTEGT,2018,4,25,M,M,1993,748L,156,56SI,...,0,0,0,1,0,0,0,0,0,0
9998,8EKC4O9,2015,5,34,F,U,1981,49BM,18,T4MS,...,0,0,0,0,0,0,0,0,0,0


In [15]:
test_x=dt.iloc[:,:10].values



In [16]:
result=np.array(model((test_x,1)).detach().cpu())

In [17]:
result

array([[0.00275231, 0.02309757, 0.00432645, ..., 0.05065098, 0.03522831,
        0.03282816],
       [0.00408793, 0.00839597, 0.00301188, ..., 0.03728336, 0.02776643,
        0.02450823],
       [0.00666784, 0.0015998 , 0.00570961, ..., 0.04200678, 0.02697119,
        0.02372017],
       ...,
       [0.00375696, 0.01088406, 0.00257171, ..., 0.03755647, 0.02780551,
        0.02407444],
       [0.1389347 , 0.3450307 , 0.1493659 , ..., 0.1475434 , 0.14327414,
        0.14478652],
       [0.05508614, 0.16868997, 0.06130209, ..., 0.08434118, 0.07887278,
        0.07749103]], dtype=float32)

In [18]:
ref=dt.iloc[:,10:].values

In [19]:
ref

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [20]:
file=open('answer.txt','w')
for i in range(len(ref)):
    for j in range(len(ref[i])):
        if(ref[i][j]==0):
            file.write(str(result[i][j])+'\n')
        else:
            file.write(str(1)+'\n')
file.close()

In [21]:
train_batch_y[0]

tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0.], device='cuda:0')

In [22]:
model.item_factors(torch.tensor(0).to(device))

tensor([ 0.5850, -0.5468,  0.4568,  0.5115, -0.3868,  0.4361, -0.5823,  0.4832,
         0.4008,  0.4638,  0.5329, -0.5070,  0.5564,  0.5547,  0.5699, -0.4921,
        -0.4130, -0.4190, -0.4624, -0.5533,  0.5182, -0.4667,  0.5293, -0.3775,
         0.4680,  0.4520, -0.5510,  0.3693, -0.5441,  0.4593,  0.5319, -0.4323,
        -0.4363,  0.4846, -0.5009,  0.4943, -0.5261, -0.4501, -0.4128, -0.5592,
         0.4350, -0.5708,  0.5176, -0.4910, -0.5183,  0.4970,  0.4476, -0.5443,
        -0.5062,  0.4309, -0.5945, -0.5537,  0.4133, -0.4575, -0.4247, -0.4226,
        -0.5412,  0.4439,  0.5536,  0.4584, -0.4902, -0.3769, -0.4883, -0.5173,
        -0.4350,  0.5217, -0.4746,  0.4159, -0.4687, -0.5065,  0.5234,  0.3979,
        -0.3594,  0.5361, -0.4550, -0.5459,  0.4178,  0.3893,  0.5144,  0.4333,
         0.3997, -0.4465,  0.4329,  0.5596, -0.4133,  0.4716, -0.5527, -0.5258,
        -0.5362,  0.4656, -0.4627, -0.5462,  0.5077, -0.5958,  0.4045,  0.4374,
         0.4894,  0.5537, -0.3802, -0.53

In [23]:
model.birth_year_biases(torch.tensor(0).to(device))+model.item_biases(torch.tensor(0).to(device))

AttributeError: 'MatrixFactorization' object has no attribute 'birth_year_biases'